In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython import display
display.set_matplotlib_formats("svg")

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
#Read in processed data
iris = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/processed_2022_test_data.csv")

In [ ]:
list(iris.columns)

['Unnamed: 0',
 'conforming_loan_limit',
 'derived_loan_product_type',
 'derived_dwelling_category',
 'derived_race',
 'derived_sex',
 'action_taken',
 'preapproval',
 'loan_type',
 'loan_purpose',
 'lien_status',
 'open_end_line_of_credit',
 'business_or_commercial_purpose',
 'loan_amount',
 'combined_loan_to_value_ratio',
 'loan_term',
 'negative_amortization',
 'interest_only_payment',
 'balloon_payment',
 'other_nonamortizing_features',
 'property_value',
 'construction_method',
 'occupancy_type',
 'manufactured_home_secured_property_type',
 'manufactured_home_land_property_interest',
 'total_units',
 'income',
 'debt_to_income_ratio',
 'applicant_age',
 'tract_population',
 'tract_minority_population_percent',
 'ffiec_msa_md_median_family_income',
 'tract_to_msa_income_percentage',
 'tract_owner_occupied_units',
 'tract_one_to_four_family_homes',
 'tract_median_age_of_housing_units',
 'longitude',
 'latitude',
 'politics']

In [ ]:
 #save race as seperate dataframe, but when testing you have to drop the column
races=pd.DataFrame(iris['derived_race'], columns=["derived_race"])
politics = pd.DataFrame(iris['politics'], columns=["politics"])
iris=iris.drop(columns=[ 'Unnamed: 0', 'politics', 'derived_race'])

### End of Data Cleaning / Testing on Model





In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=35, out_features=512)
        # self.hidden_1 = nn.Linear(in_features=256, out_features=512)
        self.output = nn.Linear(in_features=512, out_features=1)

    def forward(self, x):
        x = F.sigmoid(self.input(x))
        # x = F.sigmoid(self.hidden_1(x))
        return F.sigmoid(self.output(x))

In [ ]:
model = Net()
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/mortgage_applications_model.pt"))
model.eval()

Net(
  (input): Linear(in_features=35, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
iris.drop(iris[iris["derived_race"] =="Free Form Text Only"].index, inplace = True)

iris.loc[iris["derived_race"]=='White', "derived_race"]=0
iris.loc[iris["derived_race"]=='Black or African American', "derived_race"]=1
iris.loc[iris["derived_race"]=='Joint', "derived_race"]=1
iris.loc[iris["derived_race"]=='Asian', "derived_race"]=1
iris.loc[iris["derived_race"]=='2 or more minority races', "derived_race"]=1
iris.loc[iris["derived_race"]=='Native Hawaiian or Other Pacific Islander', "derived_race"]=1
iris.loc[iris["derived_race"]=='American Indian or Alaska Native', "derived_race"]=1
iris["derived_race"]=iris["derived_race"].astype(int)
iris["derived_race"]=iris["derived_race"].apply(pd.to_numeric)

In [ ]:
cleaned_data = iris

In [ ]:
#cleaned_data=cleaned_data.drop(columns=["derived_race"])
print(races.shape)
print(cleaned_data.shape)

(755292, 1)
(755292, 36)


In [ ]:
y = torch.tensor(
    [0 if i == 0 else 1 for i in cleaned_data["action_taken"]],
    dtype=torch.long
)

print(y.shape)

torch.Size([755292])


In [ ]:
income = torch.tensor(cleaned_data["income"], dtype = torch.long)

cleaned_data = (cleaned_data - cleaned_data.mean())/cleaned_data.std()


X = torch.tensor(cleaned_data.drop("action_taken", axis=1).values, dtype=torch.float)
print(X.shape)

print(cleaned_data.shape)

torch.Size([755292, 35])
(755292, 36)


In [ ]:
derived_sex = torch.tensor(cleaned_data['derived_sex'], dtype = torch.long)
debt_to_income = torch.tensor(iris['debt_to_income_ratio'], dtype = torch.long)

## Creating NISHE dictionary (calculated using Excel)

In [ ]:
NISHE = {
    "Asian": {
      "Income": {
          (-500,-1): 1.26,
          (0,100): 1.59,
          (101,200): 1.26,
          (801,900): 1.01,
          (901,1000): 1.07
      },
      "Debt-to-income ratio": {
          (20,30): 1.161544326,
          (31,40): 1.136782846,
          (41,50): 1.189517478,
          (51,60): 1.375935792
      },
      "Political": {
          "Republican": 1.444055527,
          "Democrat": 1.106017308,
          "Swing": 1.175289741
      },
      "Gender": {
          "Joint": 1.370066309,
          "Male": 1.333896224,
          "Female": 1.480224964
      }
    },
    "Black or African American": {
      "Income": {
          (201,300): 1.03,
          (301,400): 1.03,
          (401,500): 1.20,
          (501,600): 1.09,
          (801,900): 1.14,
          (901,1000): 1.41,
          (1000,5000): 1.20
      },
      "Debt-to-income ratio": {

      },
      "Political": {

      },
      "Gender": {

      }
    },
    "White": {
      "Income": {
          (401,500): 1.09
      },
      "Debt-to-income ratio": {

      },
      "Political": {

      },
      "Gender": {

      }
    },
    "Joint": {
      "Income": {
          (-500,-1): 1.41025641,
          (601,700): 1.136781609,
      },
      "Debt-to-income ratio": {
          (20,30): 1.161544326,
          (31,40): 1.136782846,
          (41,50): 1.189517478,
          (51,60): 1.375935792
      },
      "Political": {

      },
      "Gender": {

      }
    },
    "Native Hawaiian or Other Pacific Islander": {
      "Income": {
          (0,100): 1.12413622,
          (201,300): 1.093849253,
          (301,400): 1.093285182,
          (601,700): 1.15,
          (701,800): 1.9375,
          (1000,5000): 2.0
      },
      "Debt-to-income ratio": {

      },
      "Political": {
          "Republican": 1.03300036,
          "Swing": 1.105983928
      },
      "Gender": {
          "Joint": 1.032849322,
          "Male": 1.121450697,
          "Female": 1.029656739
      }
    },
    "American Indian or Alaska Native": {
      "Income": {
          (101,200): 1.065746993,
          (201,300): 1.023000556,
          (301,400): 1.071419478,
          (501,600): 1.247875749,
          (601,700): 1.25952381,
          (701,800): 1.085
      },
      "Debt-to-income ratio": {

      },
      "Political": {
          "Democrat": 1.069507037
      },
      "Gender": {
          "Female": 1.246410021
      }
    },
    "2 or more minority races": {
      "Income": {
          (0,100): 1.116843707,
          (101,200): 1.150093147,
          (401,500): 1.066666667,
          (501,600): 1.379231091,
          (701,800): 1.1625,
          (801,900): 1.854766187,
          (1000,5000): 1.691747573
      },
      "Debt-to-income ratio": {
          (20,30): 2.909090909,
          (31,40): 2.327244462,
          (41,50): 1.598652107,
          (51,60): 1
      },
      "Political": {
          "Republican": 1.040509259,
          "Democrat": 1.056728661,
          "Swing": 1.056728661
      },
      "Gender": {
          "Joint": 1.074808026,
          "Male": 1.053723017
      }
    }
}

In [ ]:
def NISHE_scaled_output(race, income, political_group, gender, dti_ratio):
  scale = []
  race_dict = NISHE[race]
  race_dict_income = race_dict["Income"]
  race_dict_political = race_dict["Political"]
  race_dict_gender = race_dict["Gender"]
  race_dict_dti = race_dict["Debt-to-income ratio"]

  bracket = int(income / 100)
  if bracket < 0:
    income = (-500,-1)
  elif bracket == 0:
    income = (0,100)
  elif bracket < 10:
    income = ((100 * bracket + 1), (100*(bracket+1)))
  else:
    income = (1000,5000)

  dti_ratio = int(dti_ratio / 10)
  if dti_ratio ==2:
    dti_ratio = (20,30)
  elif dti_ratio == 3:
    dti_ratio = (31,40)
  elif dti_ratio == 4:
    dti_ratio = (41,50)
  elif dti_ratio == 5:
    dti_ratio = (51,60)
  elif dti_ratio >= 6:
    dti_ratio = (61,84)

  if income in race_dict_income:
    scale.append(race_dict_income[income])
  if political_group in race_dict_political:
    scale.append(race_dict_political[political_group])
  if gender in race_dict_gender:
    scale.append(race_dict_gender[gender])
  if dti_ratio in race_dict_dti:
    scale.append(race_dict_dti[dti_ratio])

  if len(scale) > 0:
    mean = np.mean(scale)
    # if race != "White":
    #   return mean * 0.8
    return mean
  else:
    return 1.0

##Testing on Income Brackets by Race

In [ ]:
def calculate_race_approval_accuracy(race):
  approved_predictions = 0
  accurate_predictions = 0
  total_predictions = 0
  income_approvals = {
      (-500,-1): 0,
      (0,100): 0,
      (101,200): 0,
      (201,300): 0,
      (301,400): 0,
      (401,500): 0,
      (501,600): 0,
      (601,700): 0,
      (701,800): 0,
      (801,900): 0,
      (901,1000): 0,
      (1000,5000): 0
  }

  income_bracket_totals = {
      (-500,-1): 0,
      (0,100): 0,
      (101,200): 0,
      (201,300): 0,
      (301,400): 0,
      (401,500): 0,
      (501,600): 0,
      (601,700): 0,
      (701,800): 0,
      (801,900): 0,
      (901,1000): 0,
      (1000,5000): 0
  }

  for index, row in races.iterrows():
      if index >= 754423:
        break
      # print(row['derived_race'])
      if row['derived_race'] == race:
          output = model(X[index].cpu())
          # output = y[index]
          i = income[index]
          # print(i)
          bracket = int(i / 100)
          if bracket < 0:
            bracket_tuple = (-500,-1)
          elif bracket == 0:
            bracket_tuple = (0,100)
          elif bracket < 10:
            bracket_tuple = ((100 * bracket + 1), (100*(bracket+1)))
          else:
            bracket_tuple = (1000,5000)
          political_group = politics.iloc[:, 0][index]
          scale = NISHE_scaled_output(race, i, political_group, derived_sex[index], debt_to_income[index])
          # print(bracket)
          # print(output)
          output *= scale
          output = 1 if output >= 0.5 else 0

          if output == 1:
              income_approvals[bracket_tuple] += 1
              approved_predictions += 1

          income_bracket_totals[bracket_tuple] += 1

          if output == y[index]:
              accurate_predictions += 1
          total_predictions += 1

  print(total_predictions)
  print(accurate_predictions)
  print(approved_predictions)
  percentage_approval = approved_predictions / total_predictions
  percentage_accuracy = accurate_predictions / total_predictions

  print(f'Approval rate for {race} applicants: {percentage_approval * 100:.2f}%')
  print(f'Accuracy rate for {race} applicants: {percentage_accuracy * 100:.2f}%')

  for key, val in income_approvals.items():
    if income_bracket_totals[key] == 0:
      perc = "No one in bracket"
    else:
      perc = val/income_bracket_totals[key] * 100
    #print(f'Income bracket approval for bracket {key} for {race} applicants: {perc}%')
    print(perc)

In [ ]:
races['derived_race'].unique()


array(['Black or African American', 'White', 'Asian', 'Joint',
       'Native Hawaiian or Other Pacific Islander',
       'American Indian or Alaska Native', '2 or more minority races',
       'Free Form Text Only'], dtype=object)

In [ ]:
calculate_race_approval_accuracy("Asian")

58595
37929
24350
Approval rate for Asian applicants: 41.56%
Accuracy rate for Asian applicants: 64.73%
26.31578947368421
37.31003337914785
43.795938104448744
44.82354663933226
48.01980198019802
50.5008347245409
52.83993115318416
51.28205128205128
47.08994708994709
38.1294964028777
42.04545454545455
48.30097087378641


In [ ]:
calculate_race_approval_accuracy("Black or African American")

121384
68305
59047
Approval rate for Black or African American applicants: 48.64%
Accuracy rate for Black or African American applicants: 56.27%
28.205128205128204
49.60849855762626
47.89594491201224
39.02038903210687
38.78787878787879
41.41414141414141
38.46153846153847
37.81512605042017
38.75
40.816326530612244
42.22222222222222
36.771300448430495


In [ ]:
calculate_race_approval_accuracy("American Indian or Alaska Native")

9261
5299
4376
Approval rate for American Indian or Alaska Native applicants: 47.25%
Accuracy rate for American Indian or Alaska Native applicants: 57.22%
66.66666666666666
47.92984473835538
46.644295302013425
39.05723905723906
38.095238095238095
41.02564102564102
42.10526315789473
39.130434782608695
42.857142857142854
37.5
75.0
50.0


In [ ]:
calculate_race_approval_accuracy("2 or more minority races")

3419
1888
1732
Approval rate for 2 or more minority races applicants: 50.66%
Accuracy rate for 2 or more minority races applicants: 55.22%
No one in bracket
49.17175239755885
51.886792452830186
58.857142857142854
60.416666666666664
61.53846153846154
71.42857142857143
83.33333333333334
66.66666666666666
40.0
66.66666666666666
47.05882352941176


In [ ]:
calculate_race_approval_accuracy("Native Hawaiian or Other Pacific Islander")

3479
2047
1569
Approval rate for Native Hawaiian or Other Pacific Islander applicants: 45.10%
Accuracy rate for Native Hawaiian or Other Pacific Islander applicants: 58.84%
0.0
44.429766622633196
48.148148148148145
40.49079754601227
35.0
44.44444444444444
70.0
33.33333333333333
40.0
0.0
0.0
47.61904761904761


In [ ]:
calculate_race_approval_accuracy("White")

539136
324053
254206
Approval rate for White applicants: 47.15%
Accuracy rate for White applicants: 60.11%
30.54003724394786
48.55165830146357
46.74628563304187
39.79072309854755
38.23128013829497
41.11578947368421
39.406602590890095
38.983050847457626
41.71656686626746
43.27485380116959
43.9453125
43.52705410821643


In [ ]:
calculate_race_approval_accuracy("Joint")

18280
10332
9628
Approval rate for Joint applicants: 52.67%
Accuracy rate for Joint applicants: 56.52%
25.0
52.838658568787956
54.2808703023453
50.345781466113415
47.32394366197183
48.85245901639344
52.63157894736842
36.04651162790697
41.17647058823529
54.54545454545454
54.54545454545454
50.54945054945055


##Testing on Political Differences

In [ ]:
def calculate_race_approval_accuracy_politics_real(race):
  approved_predictions = 0
  total_predictions = 0
  accurate_predictions=0
  politics_approvals = {
      'republican': 0,
      'democrat': 0,
      'swing': 0,
  }

  politics_totals = {
      'republican': 0,
      'democrat': 0,
      'swing': 0,
  }

  for index, row in races.iterrows():
      if index >= 755292:
        break
      if row['derived_race'] == race:
          output = model(X[index])
          bracket = politics["politics"][index]
          scale = NISHE_scaled_output(race, income[index], bracket, derived_sex[index], debt_to_income[index])
          output *= scale
          output = 1 if output >= 0.5 else 0

          if output == 1:
            if bracket =="Republican":
              politics_approvals['republican'] += 1
            elif bracket == "Democrat":
              politics_approvals['democrat'] += 1
            elif bracket == "Swing":
              politics_approvals['swing'] += 1
            approved_predictions+=1

          if output == y[index]:
            accurate_predictions += 1

          if bracket =="Republican":
            politics_totals['republican'] += 1
          elif bracket == "Democrat":
            politics_totals['democrat'] += 1
          elif bracket == "Swing":
            politics_totals['swing'] += 1

          total_predictions += 1

  print(total_predictions)
  print(approved_predictions)
  percentage_approval = approved_predictions / total_predictions

  print(f'Approval rate for {race} applicants: {percentage_approval * 100:.2f}%')

  for key, val in politics_approvals.items():
    if politics_totals[key] == 0:
      perc = "No one in bracket"
    else:
      perc = val/politics_totals[key] * 100
    #print(f'Income bracket approval for bracket {key} for {race} applicants: {perc}%')
    print(perc)


calculate_race_approval_accuracy_politics_real("Asian")
calculate_race_approval_accuracy_politics_real("White")
calculate_race_approval_accuracy_politics_real("Black or African American")
calculate_race_approval_accuracy_politics_real("Joint")
calculate_race_approval_accuracy_politics_real("Native Hawaiian or Other Pacific Islander")
calculate_race_approval_accuracy_politics_real("2 or more minority races")
calculate_race_approval_accuracy_politics_real("American Indian or Alaska Native")


58613
24586
Approval rate for Asian applicants: 41.95%
42.802206988797856
38.71763255240444
41.73378977043898
539883
254860
Approval rate for White applicants: 47.21%
50.626318835158024
39.8951392925815
44.743917846383106
121467
59116
Approval rate for Black or African American applicants: 48.67%
51.34697482702782
42.620329855267585
48.36341788354536
18292
8651
Approval rate for Joint applicants: 47.29%
46.391369047619044
44.44592080786606
42.860623781676416
3480
1570
Approval rate for Native Hawaiian or Other Pacific Islander applicants: 45.11%
47.69874476987448
44.35102469784551
41.59891598915989
3420
1515
Approval rate for 2 or more minority races applicants: 44.30%
47.58064516129033
40.0
43.91100702576112
9268
4379
Approval rate for American Indian or Alaska Native applicants: 47.25%
51.79794520547946
39.55154157583307
44.380853277835584


## Intersectionality of race with gender

In [ ]:
def calculate_race_gender_approval(race, gender):
  approved_predictions = 0
  total_predictions = 0

  for index, row in races.iterrows():
      if index >= 755292:
        break
      # print(row['derived_race'])
      if row['derived_race'] == race and derived_sex[index] == gender:
          output = model(X[index].cpu())
          # output = y[index]
          scale = NISHE_scaled_output(race, income[index], politics.iloc[:, 0][index], gender, debt_to_income[index])
          output *= scale
          output = 1 if output >= 0.5 else 0

          if output == 1:
              approved_predictions += 1
          total_predictions += 1
  percentage_approval = approved_predictions / total_predictions

  if gender == 0:
    gender_name = "Joint"
  elif gender == 1:
    gender_name = "Male"
  else:
    gender_name = "Female"

  print(f'Approval rate for {race} {gender_name} applicants: {percentage_approval * 100:.2f}%')

In [ ]:
calculate_race_gender_approval("Black or African American", 0)
calculate_race_gender_approval("Black or African American", 1)
calculate_race_gender_approval("Black or African American", 2)

Approval rate for Black or African American Joint applicants: 48.91%
Approval rate for Black or African American Male applicants: 47.21%
Approval rate for Black or African American Female applicants: 45.79%


In [ ]:
calculate_race_gender_approval("American Indian or Alaska Native", 0)
calculate_race_gender_approval("American Indian or Alaska Native", 1)
calculate_race_gender_approval("American Indian or Alaska Native", 2)
calculate_race_gender_approval("Asian", 0)
calculate_race_gender_approval("Asian", 1)
calculate_race_gender_approval("Asian", 2)
calculate_race_gender_approval("Native Hawaiian or Other Pacific Islander", 0)
calculate_race_gender_approval("Native Hawaiian or Other Pacific Islander", 1)
calculate_race_gender_approval("Native Hawaiian or Other Pacific Islander", 2)
calculate_race_gender_approval("Joint", 0)
calculate_race_gender_approval("Joint", 1)
calculate_race_gender_approval("Joint", 2)
calculate_race_gender_approval("2 or more minority races", 0)
calculate_race_gender_approval("2 or more minority races", 1)
calculate_race_gender_approval("2 or more minority races", 2)

Approval rate for American Indian or Alaska Native Joint applicants: 46.24%
Approval rate for American Indian or Alaska Native Male applicants: 48.02%
Approval rate for American Indian or Alaska Native Female applicants: 34.55%
Approval rate for Asian Joint applicants: 39.34%
Approval rate for Asian Male applicants: 40.64%
Approval rate for Asian Female applicants: 34.06%
Approval rate for Native Hawaiian or Other Pacific Islander Joint applicants: 44.39%
Approval rate for Native Hawaiian or Other Pacific Islander Male applicants: 40.81%
Approval rate for Native Hawaiian or Other Pacific Islander Female applicants: 39.62%
Approval rate for Joint Joint applicants: 44.77%
Approval rate for Joint Male applicants: 45.66%
Approval rate for Joint Female applicants: 42.37%
Approval rate for 2 or more minority races Joint applicants: 42.74%
Approval rate for 2 or more minority races Male applicants: 44.33%
Approval rate for 2 or more minority races Female applicants: 44.90%


In [ ]:
calculate_race_gender_approval("White", 0)
calculate_race_gender_approval("White", 1)
calculate_race_gender_approval("White", 2)

Approval rate for White Joint applicants: 45.21%
Approval rate for White Male applicants: 46.16%
Approval rate for White Female applicants: 41.97%


In [ ]:
def calculate_race_approval_accuracy_debt(race):
  approved_predictions = 0
  accurate_predictions = 0
  total_predictions = 0
  debt_income_approvals = {
      (20,30): 0,
      (31,40): 0,
      (41,50): 0,
      (51,60): 0,
      (61,84): 0,
  }

  debt_income_totals = {
      (20,30): 0,
      (31,40): 0,
      (41,50): 0,
      (51,60): 0,
      (61,84): 0,
  }

  for index, row in races.iterrows():
      if index >= 755292:
        break
      if row['derived_race'] == race:
          output = model(X[index])
          i = debt_to_income[index]
          bracket = int(i / 10)
          scale = NISHE_scaled_output(race, income[index], politics.iloc[:, 0][index], derived_sex[index], i)
          output *= scale
          output = 1 if output >= 0.5 else 0

          if output == 1:
              if bracket ==2:
                debt_income_approvals[(20,30)] += 1
              elif bracket == 3:
                debt_income_approvals[(31,40)] += 1
              elif bracket == 4:
                debt_income_approvals[(41,50)] += 1
              elif bracket == 5:
                debt_income_approvals[(51,60)] += 1
              elif bracket >= 6:
                debt_income_approvals[(61,84)] += 1
              approved_predictions += 1

          if bracket ==2:
            debt_income_totals[(20,30)] += 1
          elif bracket == 3:
            debt_income_totals[(31,40)] += 1
          elif bracket == 4:
            debt_income_totals[(41,50)] += 1
          elif bracket == 5:
            debt_income_totals[(51,60)] += 1
          elif bracket >= 6:
            debt_income_totals[(61,84)] += 1

          if output == y[index]:
              accurate_predictions += 1
          total_predictions += 1

  print(total_predictions)
  print(accurate_predictions)
  print(approved_predictions)
  percentage_approval = approved_predictions / total_predictions
  percentage_accuracy = accurate_predictions / total_predictions

  print(f'Approval rate for {race} applicants: {percentage_approval * 100:.2f}%')
  print(f'Accuracy rate for {race} applicants: {percentage_accuracy * 100:.2f}%')

  for key, val in debt_income_approvals.items():
    if debt_income_totals[key] == 0:
      perc = "No one in bracket"
    else:
      perc = val/debt_income_totals[key] * 100
    #print(f'Income bracket approval for bracket {key} for {race} applicants: {perc}%')
    print(perc)


In [ ]:
calculate_race_approval_accuracy_debt("American Indian or Alaska Native")
calculate_race_approval_accuracy_debt("Asian")
calculate_race_approval_accuracy_debt("Black or African American")

calculate_race_approval_accuracy_debt("Native Hawaiian or Other Pacific Islander")
calculate_race_approval_accuracy_debt("Joint")

9268
5305
4379
Approval rate for American Indian or Alaska Native applicants: 47.25%
Accuracy rate for American Indian or Alaska Native applicants: 57.24%
48.46534653465346
67.5219446320054
71.25841532884516
56.918687589158345
9.285127362366474
58613
37942
24367
Approval rate for Asian applicants: 41.57%
Accuracy rate for Asian applicants: 64.73%
44.18103448275862
60.484699116861776
66.42043705755617
46.493079990616934
7.99650043744532
121467
68348
59116
Approval rate for Black or African American applicants: 48.67%
Accuracy rate for Black or African American applicants: 56.27%
48.7690044139284
66.08901954716592
71.30255189554381
60.73308359808431
9.930694305694306
3480
2048
1570
Approval rate for Native Hawaiian or Other Pacific Islander applicants: 45.11%
Accuracy rate for Native Hawaiian or Other Pacific Islander applicants: 58.85%
46.30434782608695
62.44979919678715
70.68062827225131
57.38255033557047
14.113597246127366
18292
10341
9640
Approval rate for Joint applicants: 52.70%
Ac

In [ ]:
calculate_race_approval_accuracy_debt("2 or more minority races")

3420
1888
1733
Approval rate for 2 or more minority races applicants: 50.67%
Accuracy rate for 2 or more minority races applicants: 55.20%
65.05636070853463
72.80701754385966
74.96740547588006
52.63157894736842
12.878048780487806


In [ ]:
calculate_race_approval_accuracy_debt("White")

539883
324551
254860
Approval rate for White applicants: 47.21%
Accuracy rate for White applicants: 60.12%
48.63863950276243
62.47795414462081
66.65036250463486
51.30968279363736
8.562248037569569
